# Lectura 55: SQL en Polars II

## Crear un `SQLContext` con la función `with`

Podemos controlar la duración del registro de una tabla utilizando `SQLContext` en conjunto con la función `with`. A menudo, esto puede resultar más útil cuando se desea este tipo de control.

In [1]:
import polars as pl

df = pl.DataFrame(
    {
        'id': [1, 2, 3],
        'color': ['rojo', 'verde', 'azul']
    }
)

lf = pl.LazyFrame(
    {
        'letra': ['a', 'b', 'c'],
        'conteo': [6, 9, 3]
    }
)

df1 = pl.DataFrame({'a': [1]})

df2 = pl.DataFrame({'b': [2]})

df3 = pl.DataFrame({'c': [3]})

En el siguiente ejemplo registraremos dos DataFrames en el momento de la construcción y otros tres dentro del scope de la función `with`.

In [2]:
with pl.SQLContext(df=df, lf=lf) as ctx:
    
    ctx.register_many(df1=df1, df2=df2, df3=df3)
    
    print(ctx.tables())

['df', 'df1', 'df2', 'df3', 'lf']


Observemos como dentro del contexto de la función `with` existían 5 tablas. Veamos que sucede si mostramos las tablas que que hay registradas ahora.

In [3]:
ctx.tables()

['df', 'lf']

Las tablas registradas dentro del scope se cancelan automáticamente al salir del scope de la función `with`. Sin embargo, las tablas registradas en la construcción persistirán en los scopes posteriores.

## Ejecutar consultas SQL a partir de diferentes fuentes

In [4]:
with pl.SQLContext(
    juegos=pl.scan_csv('./data/games.csv'),
    ligas=pl.scan_csv('./data/leagues.csv'),
    eager_execution=True
) as ctx:
    query = """
    SELECT
        name,
        count(*) AS conteo
    FROM juegos
    LEFT JOIN ligas ON competition_code = league_id
    GROUP BY name
    ORDER BY conteo DESC
    """
    
    print(ctx.execute(query))

shape: (14, 2)
┌──────────────────────┬────────┐
│ name                 ┆ conteo │
│ ---                  ┆ ---    │
│ str                  ┆ u32    │
╞══════════════════════╪════════╡
│ null                 ┆ 12367  │
│ premier-liga         ┆ 3084   │
│ serie-a              ┆ 2820   │
│ laliga               ┆ 2814   │
│ premier-league       ┆ 2809   │
│ …                    ┆ …      │
│ eredivisie           ┆ 2203   │
│ jupiler-pro-league   ┆ 1891   │
│ super-league-1       ┆ 1713   │
│ scottish-premiership ┆ 1554   │
│ superligaen          ┆ 1358   │
└──────────────────────┴────────┘


## Funciones de tabla

Polars también admite la lectura directa de CSV, Parquet, JSON e IPC directamente en una consulta SQL utilizando las funciones de tabla `read_xxx`. Veamos un ejemplo con la función `read_csv`.

In [5]:
with pl.SQLContext(eager_execution=True) as ctx:
    
    query = """
    SELECT *
    FROM read_csv('./data/games.csv')
    """
    
    print(ctx.execute(query))

shape: (42_272, 15)
┌─────────┬────────────┬────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ game_id ┆ competitio ┆ season ┆ round      ┆ … ┆ stadium    ┆ attendance ┆ referee   ┆ url       │
│ ---     ┆ n_code     ┆ ---    ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---       ┆ ---       │
│ i64     ┆ ---        ┆ i64    ┆ str        ┆   ┆ str        ┆ i64        ┆ str       ┆ str       │
│         ┆ str        ┆        ┆            ┆   ┆            ┆            ┆           ┆           │
╞═════════╪════════════╪════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ 2457642 ┆ NLSC       ┆ 2014   ┆ Final      ┆ … ┆ Johan      ┆ 42000      ┆ Danny     ┆ https://w │
│         ┆            ┆        ┆            ┆   ┆ Cruijff    ┆            ┆ Makkelie  ┆ ww.transf │
│         ┆            ┆        ┆            ┆   ┆ ArenA      ┆            ┆           ┆ ermarkt.c │
│         ┆            ┆        ┆            ┆   ┆            ┆        

## Compatibilidad

Polars no admite todas las sentencias SQL, pero sí un subconjunto de los tipos de declaraciones más comunes.

Siempre que sea posible, Polars pretende seguir las definiciones de sintaxis y el comportamiento de las funciones de PostgreSQL.

A continuación listamos algunas de las características que no están soportadas por el momento:
- INSERT
- UPDATE
- DELETE